# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [12]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
weather_data = pd.read_csv("../WeatherPy/weather_data.csv", index_col=[0])
weather_data
#weather_data.drop([''], axis=1)

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Los Llanos de Aridane,65,ES,1595120687,69,28.66,-17.92,82.00,8.05
1,Punta Arenas,99,CL,1595120689,80,-53.15,-70.92,35.01,13.87
2,Sitka,90,US,1595120690,93,57.05,-135.33,57.20,9.17
3,Kruisfontein,0,ZA,1595120691,79,-34.00,24.73,50.67,5.77
4,Bonavista,100,CA,1595120692,67,48.65,-53.11,54.45,4.79
...,...,...,...,...,...,...,...,...,...
559,Ürümqi,100,CN,1595121398,28,43.80,87.60,86.00,4.47
560,Smithers,75,CA,1595121245,77,54.78,-127.17,64.40,4.70
561,Colorado,0,BR,1595121400,63,-22.84,-51.97,67.37,8.81
562,Ballina,0,AU,1595121401,68,-28.87,153.57,66.99,6.93


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_data["Humidity"]

In [15]:
# Plot Heatmap
new_fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=3)

# Add layer
new_fig.add_layer(heat_layer)

# Display figure
new_fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 70 and 80
new_city_df = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0) & \
                                   (weather_data["Max Temp"] >= 70) & (weather_data["Max Temp"] <= 80)].dropna()

new_city_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
40,Santa Fe,0,AR,1595120736,62,-31.63,-60.70,73.99,1.99
69,Pakhtakoron,0,TJ,1595120774,33,40.16,68.75,78.60,1.05
75,Jalu,0,LY,1595120781,47,29.03,21.55,76.91,8.68
83,Granada,0,ES,1595120658,57,37.19,-3.61,77.00,4.70
125,Rio Verde de Mato Grosso,0,BR,1595120844,37,-18.92,-54.84,73.11,5.17
184,Alta Floresta,0,BR,1595120917,47,-9.88,-56.09,76.39,4.05
192,Mount Isa,0,AU,1595120927,15,-20.73,139.50,73.40,4.70
209,Goya,0,AR,1595120950,58,-29.14,-59.26,72.50,9.78
210,Yangi Marg`ilon,0,UZ,1595120951,38,40.43,71.72,75.20,9.17
248,Ucar,0,AZ,1595121002,56,40.52,47.65,77.07,1.52


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
# Create a hotel_df
hotel_df = new_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
40,Santa Fe,AR,-31.63,-60.70,
69,Pakhtakoron,TJ,40.16,68.75,
75,Jalu,LY,29.03,21.55,
83,Granada,ES,37.19,-3.61,
125,Rio Verde de Mato Grosso,BR,-18.92,-54.84,
184,Alta Floresta,BR,-9.88,-56.09,
192,Mount Isa,AU,-20.73,139.50,
209,Goya,AR,-29.14,-59.26,
210,Yangi Marg`ilon,UZ,40.43,71.72,
248,Ucar,AZ,40.52,47.65,


In [10]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [13]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
       # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 40: Santa Fe.
Closest hotel in Santa Fe is InterTower Hotel.
------------
Retrieving Results for Index 69: Pakhtakoron.
Missing field/result... skipping.
------------
Retrieving Results for Index 75: Jalu.
Closest hotel in Jalu is Jalu Hotel.
------------
Retrieving Results for Index 83: Granada.
Closest hotel in Granada is Hotel Casa 1800 Granada.
------------
Retrieving Results for Index 125: Rio Verde de Mato Grosso.
Closest hotel in Rio Verde de Mato Grosso is Hotel Serra Verde.
------------
Retrieving Results for Index 184: Alta Floresta.
Closest hotel in Alta Floresta is Hotel Mandino.
------------
Retrieving Results for Index 192: Mount Isa.
Closest hotel in Mount Isa is Abacus Motel.
------------
Retrieving Results for Index 209: Goya.
Closest hotel in Goya is Posada Las Arecas.
------------
Retrieving Results for Index 210: Yangi Marg`ilon.
Closest hotel in Yangi Marg`ilon is Hotel Mehmon Saroy.
------------
Retrieving Results for Index 248: Ucar.


In [14]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
40,Santa Fe,AR,-31.63,-60.70,InterTower Hotel
69,Pakhtakoron,TJ,40.16,68.75,
75,Jalu,LY,29.03,21.55,Jalu Hotel
83,Granada,ES,37.19,-3.61,Hotel Casa 1800 Granada
125,Rio Verde de Mato Grosso,BR,-18.92,-54.84,Hotel Serra Verde
184,Alta Floresta,BR,-9.88,-56.09,Hotel Mandino
192,Mount Isa,AU,-20.73,139.50,Abacus Motel
209,Goya,AR,-29.14,-59.26,Posada Las Arecas
210,Yangi Marg`ilon,UZ,40.43,71.72,Hotel Mehmon Saroy
248,Ucar,AZ,40.52,47.65,Stadium


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [18]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)


new_fig.add_layer(markers)
# Display figure

new_fig

Figure(layout=FigureLayout(height='420px'))